In [1]:
import setup

In [2]:
import json

In [3]:
import requests

In [4]:
from django.db.models import Q

In [5]:
from django.forms import model_to_dict

In [6]:
setup.init_django(project_name='app')

In [7]:
from services.perplexity.client import ask_perplexity_for_draft_advice

In [8]:
from services.sleeper.utils import *

In [9]:
from sleeper_bot.models import Player

In [10]:
SLEEPER_API_BASE_URL = 'https://api.sleeper.app/v1'

In [11]:
DRAFT_ID = '1131623077649801216'

In [12]:
def fetch_data(endpoint, **kwargs):
    url = f"{SLEEPER_API_BASE_URL}{endpoint.format(**kwargs)}"
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

In [13]:
def get_draft_picks():
    return fetch_data("/draft/{draft_id}/picks", draft_id=DRAFT_ID)

In [14]:
draft_picks = get_draft_picks()
last_pick_no = max(pick['pick_no'] for pick in draft_picks) if draft_picks else 0

on_roster = positions_on_roster(draft_picks)
needed_positions = position_roster_needs(on_roster)

In [15]:
print(needed_positions)

['WR', 'TE', 'K']


In [16]:
required_positions = set()
for position in needed_positions:
    if position == 'FLEX':
        required_positions.update(['RB', 'WR', 'TE']) # This is wrong
    elif position != 'BN':
        required_positions.add(position)

In [17]:
print(required_positions)

{'WR', 'K', 'TE'}


In [18]:
def get_drafted_player_ids_2(draft_picks):
    return [pick['player_id'] for pick in draft_picks]

In [19]:
if is_current_pick_mine(last_pick_no + 1, 2, 8):
        drafted_player_ids = get_drafted_player_ids_2(draft_picks)
        print(drafted_player_ids)
        position_query = Q()
        for position in required_positions:
            position_query |= Q(fantasy_positions__contains=position)
            
        available_players_queryset = Player.objects \
                        .exclude(player_id__in=drafted_player_ids) \
                        .exclude(search_rank=None) \
                        .filter(active=True) \
                        .filter(position_query) \
                        .order_by('search_rank')[:50]
        
        # recommanded_player = ask_perplexity_for_draft_advice(available_players, needed_positions)

['6786', '4034', '7547', '3321', '6794', '7564', '8155', '9509', '5859', '8146', '9221', '8150', '6813', '9493', '4866', '11628', '2133', '8112', '3198', '8144', '1466', '7543', '9226', '10859', '5850', '4984', '6803', '8205', '2449', '2216', '6904', '7569', '5872', '7526', '4046', '4881', '4983', '5846', '4039', '8138', '6819', '8136', '11632', '4035', '5012', '8130', '4018', '9758', '7525', '8151', '9997', '8137', '6770', '4217', '10236', '9229', '2309', '4137', '5849', '6790', '7611', '4199', '7553', '4066', '5892', '6801', '4950', '10229', '1479', '4037', '9502', '5927', '7528', '11624', '10222', '11620', '9756', '8139', '5967', '2749', '4988', '4981', '5937', '6845', '5848', '7588', '1426', '11635', '11583', '11637', '8228', '6130', '4663', '8154', '6804', '5248', '11589', '9488', '9508', '4033', '3294', '11604', '8183', '8110', '1264', '4227', '11533', '4666', '7839', 'SF', 'DAL', '10937', 'NYJ']


In [20]:
        available_players = list(available_players_queryset.values('full_name', 'fantasy_positions', 'search_rank', 'player_id'))
        
        # If fantasy_positions is stored as a string, split it into a list
        for player in available_players:
            if isinstance(player['fantasy_positions'], str):
                player['fantasy_positions'] = player['fantasy_positions'].split(',')
        print(available_players)

[{'full_name': 'T.J. Hockenson', 'fantasy_positions': ['TE'], 'search_rank': 86, 'player_id': '5844'}, {'full_name': 'Antonio Brown', 'fantasy_positions': ['WR'], 'search_rank': 89, 'player_id': '536'}, {'full_name': 'Julian Edelman', 'fantasy_positions': ['WR'], 'search_rank': 92, 'player_id': '344'}, {'full_name': 'Christian Watson', 'fantasy_positions': ['WR'], 'search_rank': 101, 'player_id': '8167'}, {'full_name': 'Dallas Goedert', 'fantasy_positions': ['TE'], 'search_rank': 102, 'player_id': '5022'}, {'full_name': 'Brian Thomas', 'fantasy_positions': ['WR'], 'search_rank': 105, 'player_id': '11631'}, {'full_name': 'Dalton Schultz', 'fantasy_positions': ['TE'], 'search_rank': 107, 'player_id': '5001'}, {'full_name': 'Jameson Williams', 'fantasy_positions': ['WR'], 'search_rank': 111, 'player_id': '8148'}, {'full_name': 'Courtland Sutton', 'fantasy_positions': ['WR'], 'search_rank': 114, 'player_id': '5045'}, {'full_name': 'Curtis Samuel', 'fantasy_positions': ['WR'], 'search_rank'

In [21]:
        recommanded_player = ask_perplexity_for_draft_advice(available_players, needed_positions)
        print(recommanded_player)

{"id":"8e643cf8-0961-44de-9f74-0f82d5b5fcb9","choices":[{"finish_reason":"stop","index":0,"logprobs":null,"message":{"content":"Given your team needs and the available players, I recommend drafting **Courtland Sutton (WR) - Rank: 114, ID: 5045**. Sutton is a high-ranked wide receiver who can fill one of your remaining positions.","refusal":null,"role":"assistant","function_call":null,"tool_calls":null},"delta":{"role":"assistant","content":""}}],"created":1724601056,"model":"llama-3-sonar-large-32k-online","object":"chat.completion","service_tier":null,"system_fingerprint":null,"usage":{"completion_tokens":48,"prompt_tokens":999,"total_tokens":1047}}


In [22]:
Player.objects.get(player_id='5045')

<Player: courtlandsutton>